In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import norm

import os
import time
import itertools

from glove.model import *

In [2]:
# import file names
files = os.listdir("data/")
strains = ["MS014"]

# fit gLV models

In [3]:
exp_names = []
for strain in strains:
    # import data
    df_universal = pd.read_csv("data/Universal.csv")
    df_strain = pd.concat([pd.read_csv(f"data/{fname}") for fname in files if strain in fname])
    df = pd.concat((df_universal, df_strain))
    df.sort_values(by=["Treatments", "Time"], inplace=True)

    # determine species names 
    species = df.columns.values[2:]

    # instantiate gLV fit 
    model = gLV(species, df)

    # fit to data 
    model.fit()
    
    # list of parameter names 
    param_names = []
    for s1 in species:
        for s2 in species:
            param_names += [s1+"*"+s2]
    param_names = list(species) + param_names
    
    # plot parameter distribution
    n_species = len(species)
    Avec = model.params[n_species:]
    Aij_std = np.sqrt(np.diag(model.Ainv))[n_species:]

    plt.figure(figsize=(18,18))
    # set counter for parameter std. 
    k = 0

    for i in range(n_species):
        for j in range(n_species):
            plt.subplot(n_species, n_species, k+1)
            a = np.linspace(Avec[k]-np.std(Avec), Avec[k]+np.std(Avec))
            plt.plot(a, norm.pdf(a,Avec[k],Aij_std[k]))
            plt.axvline(x=0, c='k', alpha=.5)
            k += 1
            if j == 0:
                plt.ylabel(species[i], fontsize=18)
            if i == n_species-1:
                plt.xlabel(species[j], fontsize=18)
            #plt.xlim([-2,2])

    plt.suptitle(strain, fontsize=24)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig("figures/"+strain+".pdf")
    plt.close()
    
    # compute Wald test for each parameter
    std_errors = np.sqrt(np.diag(model.Ainv))
    walds = model.params/std_errors
    wald_p_vals = 2*norm.cdf(-np.abs(walds))

    # save to df 
    df = pd.DataFrame()
    df["Param name"] = param_names
    df["Param value"] = model.params
    df["Param stdv"]  = np.sqrt(np.diag(model.Ainv))
    df["Param p-value"] = wald_p_vals
    for j, param_name in enumerate(param_names):
        df[param_name]  = model.Ainv[:, j]
    df.to_csv("params/"+strain+".csv", index=False)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Total samples: 88, Initial regularization: 1.00e-03
Loss: 15.726, Residuals: 0.043
Loss: 9.564, Residuals: 0.124
Loss: 7.973, Residuals: 0.115
Loss: 7.793, Residuals: 0.088
Loss: 7.457, Residuals: 0.076
Loss: 6.896, Residuals: 0.053
Loss: 6.338, Residuals: 0.028
Loss: 6.215, Residuals: 0.053
Loss: 6.036, Residuals: 0.027
Loss: 6.004, Residuals: 0.022
Loss: 5.948, Residuals: 0.012
Loss: 5.870, Residuals: -0.003
Loss: 5.864, Residuals: -0.000
Loss: 5.853, Residuals: -0.002
Loss: 5.834, Residuals: -0.007
Loss: 5.817, Residuals: -0.011
Loss: 5.816, Residuals: -0.011
Loss: 5.804, Residuals: -0.014
Loss: 5.793, Residuals: -0.019
Loss: 5.792, Residuals: -0.019
Loss: 5.791, Residuals: -0.018
Loss: 5.788, Residuals: -0.019
Loss: 5.788, Residuals: -0.019
Loss: 5.788, Residuals: -0.020
Loss: 5.787, Residuals: -0.020
Loss: 5.787, Residuals: -0.020
Loss: 5.786, Residuals: -0.020
Loss: 5.786, Residuals: -0.021
Loss: 5.785, Residuals: -0.021
Loss: 5.785, Residuals: -0.021
Loss: 5.785, Residuals: -0.0